In [1]:
import pandas as pd
data = pd.read_csv("ner_dataset.csv", encoding="unicode_escape")
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [2]:
data['POS'].unique()

array(['NNS', 'IN', 'VBP', 'VBN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'CC',
       'JJ', '.', 'VBD', 'WP', '``', 'CD', 'PRP', 'VBZ', 'POS', 'VBG',
       'RB', ',', 'WRB', 'PRP$', 'MD', 'WDT', 'JJR', ':', 'JJS', 'WP$',
       'RP', 'PDT', 'NNPS', 'EX', 'RBS', 'LRB', 'RRB', '$', 'RBR', ';',
       'UH', 'FW'], dtype=object)

In [3]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for idx, tok  in enumerate(vocab)}
    tok2idx = {tok:idx for idx, tok  in enumerate(vocab)}
    return tok2idx, idx2tok

token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [4]:
tag2idx

{'B-tim': 0,
 'I-art': 1,
 'I-gpe': 2,
 'I-geo': 3,
 'I-org': 4,
 'I-nat': 5,
 'I-tim': 6,
 'B-nat': 7,
 'I-per': 8,
 'B-geo': 9,
 'B-org': 10,
 'B-per': 11,
 'O': 12,
 'I-eve': 13,
 'B-eve': 14,
 'B-art': 15,
 'B-gpe': 16}

In [5]:
# Ánh xạ từng 'Word' trong DataFrame 'data' sang chỉ số tương ứng trong từ điển 'token2idx'.
data['Word_idx'] = data['Word'].map(token2idx)

# Ánh xạ từng 'Tag' trong DataFrame 'data' sang chỉ số tương ứng trong từ điển 'tag2idx'.
data['Tag_idx'] = data['Tag'].map(tag2idx)

# Điền các giá trị NaN trong DataFrame 'data' bằng giá trị hợp lệ liền kề trước đó.
# Điều này giữ cho thông tin của một câu liên tục không bị gián đoạn bởi các giá trị thiếu.
data_fillna = data.fillna(method='ffill', axis=0)

# Nhóm dữ liệu theo 'Sentence #' để có thể xử lý từng câu một cách độc lập.
# 'as_index = False' giữ cho DataFrame sau khi nhóm không sử dụng 'Sentence #' làm chỉ số.
# Với mỗi nhóm, chuyển các cột 'Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx' thành danh sách.
# Điều này tạo ra một cấu trúc dữ liệu mà mỗi câu là một mảng của từ, tags, và chỉ số tương ứng.
data_group = data_fillna.groupby(
    ['Sentence #'], as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))


C:\Users\HP\AppData\Local\Temp\ipykernel_13300\265101333.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


In [8]:
data_group

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[12297, 3547, 8144, 6881, 14740, 29979, 1501, ...","[12, 12, 12, 12, 12, 12, 9, 12, 12, 12, 12, 12..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[34597, 12037, 27460, 3574, 8517, 33454, 16782...","[16, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[14994, 15873, 11940, 19289, 18894, 25898, 305...","[12, 12, 0, 12, 12, 12, 12, 12, 9, 12, 12, 12,..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[29020, 19989, 9612, 20576, 24199, 9655, 14486...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[15741, 8769, 31570, 11281, 31282, 15944, 408,...","[9, 12, 12, 11, 8, 12, 0, 12, 9, 12, 16, 12, 1..."
...,...,...,...,...,...,...
47954,Sentence: 9995,"[Opposition, leader, Mir, Hossein, Mousavi, ha...","[NNP, NN, NNP, NNP, NNP, VBZ, VBN, PRP, VBZ, T...","[O, O, O, B-per, I-per, O, O, O, O, O, O, O, O...","[20663, 14313, 10126, 15810, 8844, 23572, 1594...","[12, 12, 12, 11, 8, 12, 12, 12, 12, 12, 12, 12..."
47955,Sentence: 9996,"[On, Thursday, ,, Iranian, state, media, publi...","[IN, NNP, ,, JJ, NN, NNS, VBN, DT, NN, IN, DT,...","[O, B-tim, O, B-gpe, O, O, O, O, O, O, O, O, B...","[11506, 11522, 14927, 34597, 24915, 30443, 419...","[12, 0, 12, 16, 12, 12, 12, 12, 12, 12, 12, 12..."
47956,Sentence: 9997,"[Following, Iran, 's, disputed, June, 12, elec...","[VBG, NNP, POS, JJ, NNP, CD, NNS, ,, NNS, NNS,...","[O, B-geo, O, O, B-tim, I-tim, O, O, O, O, O, ...","[32743, 3188, 15305, 18740, 33514, 31075, 1585...","[12, 9, 12, 12, 0, 6, 12, 12, 12, 12, 12, 12, ..."
47957,Sentence: 9998,"[Since, then, ,, authorities, have, held, publ...","[IN, RB, ,, NNS, VBP, VBN, JJ, NNS, IN, DT, VB...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[18382, 28324, 14927, 29503, 6881, 6622, 17722...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1..."


In [23]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):
    
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))
    
    tokens = data_group['Word_idx'].to_list()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value=n_token -1)
    
    tags = data_group['Tag_idx'].to_list()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value=tag2idx['O'])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)
    
    print(
    'train_tokens length:', len(train_tokens),
    '\ntrain_tokens length:', len(train_tokens),
    '\ntest_tokens length:', len(test_tokens),
    '\ntest_tags:', len(test_tags),
    '\nval_tokens:', len(val_tokens),
    '\nval_tags:', len(val_tags),
)
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

In [45]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
np.random.seed(1)
tensorflow.random.set_seed(2)

In [49]:
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [46]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim  = 64
input_length = max([len(s) for s in data_group['Word_idx'].to_list()])
n_tags = len(tag2idx)

In [58]:
def get_bilstml_lstml_model():
    model = Sequential()
    #Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
    #Bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    #LSTM layer
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    #timeDistributed layer
    model.add(TimeDistributed(Dense(n_tags, activation='relu')))
    #Optimizer
    adam = tensorflow.keras.optimizers.Adam(lr=0.0005, beta_1=0.2, beta_2=0.999)
    
    #Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [60]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        #fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose = 1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
        print("Done 1 epoch")
    return loss

In [61]:
results = pd.DataFrame()
model_bilstm_lstml = get_bilstml_lstml_model()
plot_model(model_bilstm_lstml)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstml)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 104, 64)           2251456   
                                                                 
 bidirectional_5 (Bidirecti  (None, 104, 128)          66048     
 onal)                                                           
                                                                 
 lstm_11 (LSTM)              (None, 104, 64)           49408     
                                                                 
 time_distributed_5 (TimeDi  (None, 104, 17)           1105      
 stributed)                                                      
                                                                 
Total params: 2368017 (9.03 MB)
Trainable params: 2368017 (9.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
You must install

KeyboardInterrupt: 